# Graph Algorithms

## 1 - 图的表示

以下图为例：
<img src="images/graph/graph1.png" style="height:100px;">

#### 邻接表（Adjacency List）

In [1]:
# 嵌套列表的下标对应顶点的编号
# 注：设置一个下标为0的空列表，是为了使结点编号与下标相对应，方便检索
adj_list = [[], [2], [1, 3], [2], [1]]

#### 邻接矩阵 (Adjacency Matrix)

In [2]:
# 矩阵的纵坐标和横纵标均对应顶点的编号
adj_matrix = [[0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 1, 0, 1, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0]]

- 相比邻接矩阵，邻接表占内存小，但不能快速判断一条边的存在。
- 邻接表适用于稀疏图，邻接矩阵适用于稠密图。由于实际情况中稀疏图更为常见，所以大部分情况我们都使用邻接表。

## 2 - 图的搜索

### 2.1 - 深度优先搜索 (Depth First Search, DFS)

In [3]:
# 用深度优先的顺序访问一个顶点的所有出发边
def Explore(adj_list, visited, u):
    visited[u] = True
    for v in adj_list[u]:
        if not visited[v]:
            Explore(adj_list, visited, v)

例：对下图，判断1是否能到达3和4。
<img src="images/graph/graph1.png" style="height:100px;">

In [4]:
# test case
adj_list = [[], [2], [1, 3], [2], []]
visited = [False] * (len(adj_list) + 1)
Explore(adj_list, visited, 1)
print("Is vertex-4 reacheable for vertex-1? " + str(visited[4]))
print("Is vertex-3 reacheable for vertex-1? " + str(visited[3]))

Is vertex-4 reacheable for vertex-1? False
Is vertex-3 reacheable for vertex-1? True


In [5]:
# 深度优先搜索（搜索图中所有结点）
def DFS(adj_list):
    n = len(adj_list)
    visited = [False] * n
    for u in range(n):
        if not visited[u]:
            Explore(adj_list, visited, u)

深度优先搜索：

- 时间复杂度：O(V+E)
- 空间复杂度：O(V)

#### 应用1：计算一个图形的连通区数量

连通区 (connected components)：图形中所有相连的顶点构成一个连通区。例如，下图中有4个连通区。
<img src="images/graph/cc.png" style="height:200px;">

In [6]:
# 计算一个图形的连通区数量
def NumCCs(adj_list):
    visited = [False] * len(adj_list)
    num_CC = 0
    for u in range(1, len(adj_list)):
        if not visited[u]:
            Explore(adj_list, visited, u)
            num_CC += 1
    return num_CC

In [7]:
# test case
adj_list = [[], [2], [1, 3], [2], []]
print(NumCCs(adj_list))

2


<img src="images/graph/graph1.png" style="height:100px;">

#### 应用2：记录每个结点的访问顺序

In [8]:
class VisitOrder:
    def __init__(self, adj_list):
        self.clock = 1
        self.adj_list = adj_list
        self.preorder = [-1] * len(adj_list)
        self.postorder = [-1] * len(adj_list)
    
    def Explore(self, u):
        self.preorder[u] = self.clock
        self.clock += 1
        for v in self.adj_list[u]:
            # preorder[u]<0表示该结点还未被访问
            if self.preorder[v] < 0:
                self.Explore(v)
        self.postorder[u] = self.clock
        self.clock += 1
    
    def DFS(self):
        for u in range(1, len(self.adj_list)):
            if self.preorder[u] < 0:
                self.Explore(u)
        return self.preorder, self.postorder

In [9]:
# test case
adj_list = [[], [2], [3], [1], [1]]
S = VisitOrder(adj_list)
print(S.DFS())

([-1, 1, 2, 3, 7], [-1, 6, 5, 4, 8])


<img src="images/graph/graph2.png" style="height:100px;">

#### 应用3：判断有向无环图

- 用DFS遍历图中的顶点，每个顶点有三种状态：未被访问过（0）、已被访问但还未访问其邻接点（1）、该顶点及其邻接点全被访问了（2）。这三种状态分别用0、1、2储存在visited数组中。
- 存在环的情况可定义为：在遍历过程中，某个顶点的一条边指向了状态为1的顶点。

In [10]:
# 判断一个图是否是有向无环图
class IsDAG:
    def __init__(self, adj_list):
        self.adj_list = adj_list
        self.visited = [0] * len(adj_list)

    def Explore(self, u):
        global is_dag
        self.visited[u] = 1
        for v in self.adj_list[u]:
            if self.visited[v] == 1:
                is_dag = False
                break
            elif self.visited[v] == 0:
                self.Explore(v)
        self.visited[u] = 2

    def DFS(self):
        global is_dag
        is_dag = True
        for u in range(1, len(self.adj_list)):
            if self.visited[u] == 0:
                self.Explore(u)
                if not is_dag:
                    return False
        return True

In [11]:
# test case
adj_list = [[], [2], [3], [1], [1]]
S = IsDAG(adj_list)
print(S.DFS())

False


<img src="images/graph/graph2.png" style="height:100px;">

#### 应用4：拓扑排序（只针对有向无环图）
拓扑排序：拓扑排序是图中所有结点的一种线性次序，该次序满足如下条件：如果图G包含边(u, v)，则结点u排在结点v的前面（如果图G包含环路，则不可能排出一个线性次序）。

策略：
- 用DFS遍历图中所有结点，对边(u, v)，要先结束v的访问后，才能结束u的访问。
- 记录每个结点结束访问的先后顺序，然后按逆序输出这个次序，这个次序即为所有结点的拓扑排序。

In [12]:
# 拓扑排序
class TopoSort:
    def __init__(self, adj_list):
        self.adj_list = adj_list
        self.visited = [False] * len(adj_list)
        self.post = []
    
    def Explore(self, u):
        self.visited[u] = True
        for v in self.adj_list[u]:
            if not self.visited[v]:
                self.Explore(v)
        self.post.append(u)
    
    def TopoSort(self):
        for u in range(1, len(self.adj_list)):
            if not self.visited[u]:
                self.Explore(u)
        return self.post[::-1]

In [13]:
# test case 1
adj_list = [[], [2], [], [1], [1]]
S = TopoSort(adj_list)
print(S.TopoSort())

[4, 3, 1, 2]


<img src="images/graph/graph3.png" style="height:100px;">

In [14]:
# test case 2
adj_list2 = [[], [], [1], [2, 1], [3, 1], [2, 3]]
S = TopoSort(adj_list2)
print(S.TopoSort())

[5, 4, 3, 2, 1]


<img src="images/graph/graph4.png" style="height:100px;">

#### 应用5：强连通区（只针对有向图）

- 连通性：对有向图中的两个顶点v和w，如果有边从v→w，也有边从w→v，我们就称v和w是连通的（connected）。

- 强连通区（Strongly Connected Components, SCC）：一个有向图可以被划分为若干个强连通区（Strongly connected components ），每个强连通区内的顶点都是相互连通（connected）的。例如，下图中有5个强连通区。
<img src="images/graph/scc.png" style="height:150px;">

- 有向无环图中有两种特殊的顶点：
    - 源（source）：没有入边的顶点
    - 漏（sink）：没有出边的顶点

策略：
用DFS遍历图像并记录每个结点的访问顺序，则源结点一定是最后结束访问的，即其postorder最大。
1. 将图G转置，得到G_reverse，则G中的sink变成了G_reverse中的source。
2. 第一次DFS：对G_reverse进行拓扑排序。
3. 第二次DFS：按照步骤2得到的顺序，给G划分强连通区。

In [15]:
class StronglyConnectedComponents:
    def __init__(self, adj_list):
        self.graph = adj_list
        self.visited = [0] * len(adj_list)
        self.postorder = []
    
    def ReverseGraph(self):
        graph_reverse = [[] for _ in range(len(adj_list))]
        for u in range(1, len(adj_list)):
            for v in adj_list[u]:
                graph_reverse[v].append(u)
        return graph_reverse
    
    def Explore_topo(self, u, graph_reverse):
        self.visited[u] = 1
        for v in graph_reverse[u]:
            if self.visited[v] == 0:
                self.Explore_topo(v, graph_reverse)
        self.postorder.append(u)
    
    def Explore_scc(self, u, SCC):
        self.visited[u] = 2
        for v in self.graph[u]:
            if self.visited[v] == 1:
                SCC.append(v)
                self.Explore_scc(v, SCC)
        return SCC
    
    def SCCs(self):
        SCCs = []
        graph_reverse = self.ReverseGraph()
        # 第一次DFS：拓扑排序
        for u in range(1, len(self.graph)):
            if self.visited[u] == 0:
                self.Explore_topo(u, graph_reverse)
        # 第二次DFS：划分强连通区
        for u in self.postorder[::-1]:
            if self.visited[u] == 1:
                SCC = self.Explore_scc(u, [u])
                SCCs.append(SCC)
        return SCCs

In [16]:
adj_list = [[], [2], [3], [1], [1]]
S = StronglyConnectedComponents(adj_list)
print(S.SCCs())

[[1, 2, 3], [4]]


<img src="images/graph/graph2.png" style="height:100px;">

### 2.2 - 广度优先搜索

In [17]:
from queue import Queue

def BFS(adj_list, start):
    dist = [float('inf')] * len(adj_list)
    dist[start] = 0
    q = Queue() # 用队列储存已发现但还未处理的结点
    q.put(start)
    while not q.empty():
        u = q.get()
        for v in adj_list[u]:
            if dist[v] == float('inf'):
                q.put(v)
                dist[v] = dist[u] + 1
    return dist

In [18]:
# test case
adj_list = [[], [2], [1, 3], [2], []]
print(BFS(adj_list, 1))

[inf, 0, 1, 2, inf]


<img src="images/graph/graph1.png" style="height:100px;">

广度优先搜索：

- 时间复杂度：O(V+E)
- 空间复杂度：O(V)

#### 应用1：最短路径树

BFS算法可以记录每个顶点到起点S的距离（距离即为两个顶点之间的最短路径的长度），只要在此基础上记录每个顶点的前驱结点，我们就得到一棵以S为根节点的最短路径树，每个结点的深度表示其与S的距离。

In [19]:
from queue import Queue
def ShortestPathTree(adj_list, start):
    dist = [float('inf')] * len(adj_list)
    dist[start] = 0
    prev = [None] * len(adj_list)
    q = Queue() # 用队列储存已发现但还未处理的结点
    q.put(start)
    
    while not q.empty():
        u = q.get()
        for v in adj_list[u]:
            if dist[v] == float('inf'):
                q.put(v)
                dist[v] = dist[u] + 1
                prev[v] = u
                
    return dist, prev

In [20]:
# 重构两点之间的最短路径
def ReconstructPath(start, end, prev):
    path = []
    while end != start:
        path.append(end)
        end = prev[end]
    return path[::-1]

#### 应用2：判断一张图是否是二分图（bipartite）

二分图又称作二部图，是图论中的一种特殊模型。用两种颜色给图中的顶点上色，如果任意一条边的两个顶点颜色都互不相同，那么该图为二分图。

策略：

- 用BFS遍历图像，用数组dist记录每个顶点到起点的距离。
- 遍历每个顶点u的相邻结点v：
    - 如果|dist[u] - dist[v]|为奇数，没问题；
    - 如果|dist[u] - dist[v]|为偶数，该图像不是二分图。

In [21]:
from queue import Queue
def IsBipartite(adj_list):
    dist = [float('inf')] * len(adj_list)
    q = Queue()
    q.put(1)
    dist[1] = 0
    while not q.empty():
        u = q.get()
        for v in adj_list[u]:
            if dist[v] == float('inf'):
                dist[v] = dist[u] + 1
                q.put(v)
            elif (dist[u] - dist[v]) % 2 == 0:
                return False
    return True 

In [22]:
# test case 1
adj_list = [[], [2, 3, 4], [1, 3], [1, 2], [1]]
print(IsBipartite(adj_list))

False


<img src="images/graph/graph5.png" style="height:100px;">

In [23]:
# test case 2
adj_list = [[], [4], [4, 5], [4], [1, 2, 3], [2]]
print(IsBipartite(adj_list))

True


<img src="images/graph/graph6.png" style="height:100px;">

## 3 - 最短路径

### 3.1 - 松弛操作

In [24]:
def Relax(dist, adj_list, prev, u, v):
    # u 和 v 分别是一条边的起点和终点
    if dist[v] > dist[u] + adj_list[u][v]:
        dist[v] = dist[u] + adj_list[u][v]
        prev[v] = u

### 3.2 - Bellman-Ford算法
Bellman-Ford算法可以计算源结点到其它每一个可到达的结点的距离，但前提是**源结点不能到达负权环**。

In [25]:
def BellmanFord(adj_list):
    dist = [float('inf')] * len(adj_list)
    dist[1] = 0
    for _ in range(len(adj_list) - 2):
        # 对每条边进行松弛操作
        for u in range(1, len(adj_list)):
            for v, w in adj_list[u]:
                if dist[v] > dist[u] + w:
                    dist[v] = dist[u] + w
    # 检测源结点是否能到达负权环
    for u in range(1, len(adj_list)):
        for v, w in adj_list[u]:
            if dist[v] > dist[u] + w:
                return False
    return dist

In [26]:
# test case
adj_list = [[], [(2, -5)], [(3, 2)], [(1, 1)], [(1, 2)]]
print(BellmanFord(adj_list))

False


<img src="images/graph/graph7.png" style="height:100px;">

#### 算法改进

不用循环V-1次，只要某次循环各个顶点的距离都没有发生改变，就可以跳出循环。

In [27]:
def BellmanFord2(adj_list):
    dist = [float('inf')] * len(adj_list)
    dist[1] = 0
    i = 1
    for _ in range(len(adj_list) - 2):
        is_change = False
        # 对每条边进行松弛操作
        for u in range(1, len(adj_list)):
            for v, w in adj_list[u]:
                if dist[v] > dist[u] + w:
                    dist[v] = dist[u] + w
                    is_change = True
        if not is_change:
            return dist
    # 检测源结点是否能到达负权环
    for u in range(1, len(adj_list)):
        for v, w in adj_list[u]:
            if dist[v] > dist[u] + w:
                return False
    return dist

#### 找到构成负权环的结点

In [28]:
# 找到构成负权环的结点
def NegativeCycle(adj_list):
    dist = [float('inf')] * len(adj_list)
    dist[1] = 0
    prev = [0] * len(adj_list)
    for _ in range(len(adj_list) - 2):
        # 对每条边进行松弛操作
        for u in range(1, len(adj_list)):
            for v, w in adj_list[u]:
                if dist[v] > dist[u] + w:
                    dist[v] = dist[u] + w
                    prev[v] = u
    # 找到构成负权环的结点
    cycles = []
    in_cycle = [False] * len(adj_list)
    for u in range(1, len(adj_list)):
        for v, w in adj_list[u]:
            if not in_cycle[v] and dist[v] > dist[u] + w:
                in_cycle[v] = True
                one_cycle = [v]
                x = prev[v]
                while x != v:
                    in_cycle[x] = True
                    one_cycle.append(x)
                    x = prev[x]
                cycles.append(one_cycle)
    return cycles

In [29]:
# test case
adj_list = [[], [(2, -5)], [(3, 2)], [(1, 1)], [(1, 2)]]
print(NegativeCycle(adj_list))

[[2, 1, 3]]


<img src="images/graph/graph7.png" style="height:100px;">

#### 应用：无限套利

所有源结点能经负权环到达的结点都能实现无限套利。

In [30]:
# 找到所有能实现无限套利的结点
from queue import Queue
def InfiniteArbitrage(adj_list):
    dist = [float('inf')] * len(adj_list)
    dist[1] = 0
    for _ in range(len(adj_list) - 2):
        for u in range(1, len(adj_list)):
            for v, w in adj_list[u]:
                if dist[v] > dist[u] + w:
                    dist[v] = dist[u] + w
    
    q = Queue()
    for u in range(1, len(adj_list)):
        for v, w in adj_list[u]:
            if dist[v] > dist[u] + w:
                q.put(v)
    
    # BFS
    visited = [False] * len(adj_list)
    res = []
    while not q.empty():
        u = q.get()
        res.append(u)
        visited[u] = True
        for v, _ in adj_list[u]:
            if not visited[v]:
                q.put(v)
            
    return res

In [31]:
# test
adj_list = [[], [(2, 10), (3, 100)], [(3, 5)], [(5, 7)], [(3, -18)], [(4, 10)], [(1, -1)]]
print(InfiniteArbitrage(adj_list))

[3, 5, 4]


<img src="images/graph/graph8.png" style="height:100px;">

### 3.3 - Dijkstra算法（权重>=0）

Dijkstra算法采用贪心策略，每次循环都只处理距离最小的顶点：
1. 用最小堆H储存每个结点的距离。
2. 对起点S能到达的顶点进行松弛操作。
3. 从H中提出距离最小的顶点，对它能到达的顶点进行松弛操作。
4. 重复步骤c，直到每个顶点的距离都确定为止。

In [32]:
import heapq
def Dijkstra(adj_list, start, end):
    dist = [float('inf')] * len(adj_list)
    dist[start] = 0
    processed = [False] * len(adj_list)
    # 用最小堆辅助找到距离最小的顶点
    min_heap = []
    # 堆里顶点的储存格式为(dist[v], v)
    heapq.heappush(min_heap, (0, start))
    while min_heap:
        u_dist, u = heapq.heappop(min_heap)
        if not processed[u]:
            processed[u] = True
            for v, w in adj_list[u]:
                if dist[v] > dist[u] + w:
                    dist[v] = dist[u] + w
                    heapq.heappush(min_heap, (dist[v], v))
    return dist[end]

In [33]:
# test case 1：下图中1→3的最短路径为3
adj_list = [[], [(2, 1), (3, 5)], [(3, 2)], [], [(1, 2)]]
print(Dijkstra(adj_list, 1, 3))

3


<img src="images/graph/graph9.png" style="height:100px;">

In [34]:
# test case 2: 下图中，1→5的最短路径为6
adj_list = [[], [(2, 4), (3, 2)], [(3, 3), (4, 2), (5, 3)], [(2, 1), (4, 4), (5, 4)], [], [(4, 1)]]
print(Dijkstra(adj_list, 1, 5))

6


<img src="images/graph/graph10.png" style="height:100px;">

In [35]:
# test case 3: 下图中，3→2无路径
adj_list = [[], [(2, 7), (3, 5)], [(3, 2)], []]
print(Dijkstra(adj_list, 3, 2))

inf


<img src="images/graph/graph11.png" style="height:100px;">

## 4 - 最小生成树

### 4.1 - Kruskal's Algorithm

In [36]:
class Kruskal:
    def __init__(self, n, edges):
        self.n = n
        self.edges = edges
        self.parent = [i for i in range(n)]
        self.rank = [0] * n
    
    def Find(self, i):
        if self.parent[i] != i:
            self.parent[i] = self.Find(self.parent[i])
        return self.parent[i]
    
    def Union(self, i, j):
        i_id = self.Find(i)
        j_id = self.Find(j)
        
        if i_id == j_id:
            return 
        if self.rank[i_id] > self.rank[j_id]:
            self.parent[j_id] = i_id
        else:
            self.parent[i_id] = j_id
            if self.rank[i_id] == self.rank[j_id]:
                self.rank[j_id] += 1
    
    def Kruskal(self):
        mst = []
        min_len = 0
        self.edges.sort(key=lambda x:x[2])
        for u, v, w in edges:
            if len(mst) == self.n - 1:
                break
            if self.Find(u) != self.Find(v):
                self.Union(u, v)
                min_len += w
                mst.append((u, v, w))
        return mst, min_len

In [37]:
# test
edges = [(0, 1, 2.0), (0, 2, 1.4), (0, 3, 3.0), (0, 4, 3.6), 
         (1, 2, 1.4), (1, 3, 3.6), (1, 4, 3.0), (2, 3, 2.2),
         (2, 4, 2.2), (3, 4, 2.0)]
S = Kruskal(5, edges)
print(S.Kruskal())

([(0, 2, 1.4), (1, 2, 1.4), (3, 4, 2.0), (2, 3, 2.2)], 7.0)


### 4.2 - Prims's Algorithm

In [38]:
import heapq
def Prim(adj_list):
    cost = [float('inf')] * len(adj_list)
    parent = [None] * len(adj_list)
    visited = [False] * len(adj_list)
    cost[1] = 0
    min_heap = []
    heapq.heappush(min_heap, (0, 1))
    while min_heap:
        _, u = heapq.heappop(min_heap)
        if not visited[u]:
            visited[u] = True
            for v, w in adj_list[u]:
                if not visited[v] and w < cost[v]:
                    parent[v] = u
                    cost[v] = w
                    heapq.heappush(min_heap, (cost[v], v))
    # 计算最小生成树的总权重和
    length = 0
    for i in range(1, len(adj_list)):
        length += cost[i]
    return parent, length

In [39]:
adj_list = [[], [(2, 2), (3, 1.4), (4, 3)], [(1, 2), (3, 1.4), (5, 3)], [(1, 1.4), (2, 1.4), (4, 2.2), (5, 2.2)], 
            [(1, 3), (3, 2.2), (5, 2)], [(2, 3), (3, 2.2), (4, 2)]]
print(Prim(adj_list))

([None, None, 3, 1, 3, 4], 7.0)
